<a href="https://colab.research.google.com/github/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation/blob/main/Explainability%20models/LIME_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install eli5
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 10.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 KB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107747 sha256=c7fd8e4f52b6da4ce902cb91d1bd6dc0fc695c4fff488fa59f1d24cd37776538
  Stored in directory: /root/.cache/pip/wheels/7b/26/a5/8460416695a992a2966b41caa5338e5e7fcea98c9d032d055c
Successfully built eli5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import scipy
import numpy as np
from scipy import stats

#error =  cannot import itemfreq from scipyl.stats
# below code solved the issue 
def monkeypath_itemfreq(sampler_indices):
   return zip(*np.unique(sampler_indices, return_counts=True))
scipy.stats.itemfreq=monkeypath_itemfreq

import eli5
from eli5.lime import TextExplainer
from typing import List
from eli5.lime.samplers import MaskingTextSampler, MaskingTextSamplers
from transformers import AutoTokenizer
from IPython.core.display import HTML, display


import torch 
device =torch.device('cuda')

In [12]:
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL_split.pt',map_location=torch.device('cuda'))

In [6]:
tokenizer = AutoTokenizer.from_pretrained('google/muril-base-cased')

In [11]:
def data_cleaning(text):
  """ Removing emojis, mentions, english words, characters, space, 
  websites, urls, punctuations, etc."""
 
  text  = re.sub(r"[\'$()+=%{}~'.!:?&*-_\"\/]",r'', str(text))   # Remove Punctuations
  text  = str(text).replace(' +', ' ')
  text  = text.split()
  text = " ".join(text)

  return text

In [7]:
def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      #m = model(**inputs)
      #print(model(**inputs).logits)
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  #predicted_class_id = logits.argmax().item()
  # classifications = ['Introduction','Depression','Grey Area']
  # classification = logits.argmax().item()
  # class_name = classifications[classification]
  
  return scores


In [8]:
text = 'मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा हुई और पली-बढ़ी'

In [9]:
te = TextExplainer( random_state=42,sampler=MaskingTextSampler())

In [10]:
# Show the explanation with the actual words
te.fit(text, adapter)
te.explain_prediction(target_names=['Introduction','Depression','Grey Area'])

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Explanation(estimator="SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x7FB898176A40)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='Introduction', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x22', weight=0.733799330420781, std=None, value=2.0), FeatureWeight(feature='x11', weight=0.6118044940888885, std=None, value=5.0), FeatureWeight(feature='x25', weight=0.38145292984887386, std=None, value=2.0), FeatureWeight(feature='x29', weight=0.2766846155141516, std=None, value=1.0), FeatureWeight(feature='x31', weight=0.26589263031576593, std=None, value=1.0), FeatureWeight(feature='x20', weight=0.2522242955116597, std=None, value=1.0), FeatureWeight(feature='x4', weight=0.24644219501679748, std=None, value=1.0), FeatureWeight(feature='x8', weight=0.1909390572620481, std=None, value=1.0), FeatureWeight(feature='x28', weight=0.1851290894510395, std=None, value=1.0), FeatureWeight(feature='x15', weight=0.13258151001788504, std=None, value=1.0), FeatureWeight(feature='x16', weight=0.13025062529479384, std=None, value=1.0), FeatureWeight(feature='x32', weight=0.1274650389098524, std=None, value=1.0), FeatureWeight(feature='x0', weight=0.11459446442507873, std=None, value=1.0), FeatureWeight(feature='x21', weight=0.09198650421168678, std=None, value=1.0), FeatureWeight(feature='x2', weight=0.07906496986035977, std=None, value=1.0), FeatureWeight(feature='x5', weight=0.01826480571395648, std=None, value=1.0), FeatureWeight(feature='x7', weight=0.016047150860110736, std=None, value=1.0)], neg=[FeatureWeight(feature='x6', weight=-0.4203546139178035, std=None, value=2.0), FeatureWeight(feature='x19', weight=-0.3819416292792256, std=None, value=1.0), FeatureWeight(feature='x30', weight=-0.38004060121150435, std=None, value=1.0), FeatureWeight(feature='x27', weight=-0.27832216668742576, std=None, value=1.0), FeatureWeight(feature='x33', weight=-0.20341572285452558, std=None, value=1.0), FeatureWeight(feature='x24', weight=-0.16804569802754862, std=None, value=1.0), FeatureWeight(feature='x14', weight=-0.14873854639790307, std=None, value=1.0), FeatureWeight(feature='x9', weight=-0.1484511780560055, std=None, value=1.0), FeatureWeight(feature='x3', weight=-0.1430512879342296, std=None, value=1.0), FeatureWeight(feature='x23', weight=-0.09824176026066474, std=None, value=1.0), FeatureWeight(feature='x1', weight=-0.09168623239824567, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=-0.05462975182429233, std=None, value=1.0), FeatureWeight(feature='x12', weight=-0.04768553803383901, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.733673597671073, score=1.2900189798405162, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा हुई और पली-बढ़ी', spans=[], preserve_density=False, vec_name=None)], other=FeatureWeights(pos=[FeatureWeight(feature='x22', weight=0.733799330420781, std=None, value=2.0), FeatureWeight(feature='x11', weight=0.6118044940888885, std=None, value=5.0), FeatureWeight(feature='x25', weight=0.38145292984887386, std=None, value=2.0), FeatureWeight(feature='x29', weight=0.2766846155141516, std=None, value=1.0), FeatureWeight(feature='x31', weight=0.26589263031576593, std=None, value=1.0), FeatureWeight(feature='x20', weight=0.2522242955116597, std=None, value=1.0), FeatureWeight(feature='x4', weight=0.24644219501679748, std=None, value=1.0), FeatureWeight(feature='x8', weight=0.1909390572620481, std=None, value=1.0), FeatureWeight(feature='x28', weight=0.1851290894510395, std=None, value=1.0), FeatureWeight(feature='x15', weight=0.13258151001788504, std=None, value=1.0), FeatureWeight(feature='x16', weight=0.13025062529479384, std=None, value=1.0), FeatureWeight(feature='x32', weight=0.1274650389098524, std=None, value=1.0), FeatureWeight(feature='x0', weight=0.11459446442507873, std=None, value=1